# Creating and running a graph







In [1]:
import tensorflow as tf

x= tf.Variable(3, name="x")

y = tf.Variable(4, name = "y")

f = x*x*y +y +2



In [2]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [3]:
sess = tf.Session()

sess.run(x.initializer)
sess.run(y.initializer)

result = sess.run(f)

print(result)

42


In [4]:
sess.close()

In [5]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    

In [6]:
result


42

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    
    

In [11]:
(result)

42

In [12]:
init = tf.global_variables_initializer()



In [13]:
sess= tf.InteractiveSession()

init.run()

result = f.eval()

print(result)

42


# Managing Graphs


In [23]:

import numpy as np
def reset_graph(seed=42):


    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [32]:
reset_graph()

In [34]:
w= tf.constant(3)

x = w+2

y = x+5

z=x*3



with tf.Session() as sess:
    print(x.eval())
    print(y.eval())
    print(z.eval())
    

5
10
15


In [35]:
with tf.Session() as sess: 
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15








# Linear Regression

## Using the Normal Equation


In [61]:
reset_graph()


import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

m, n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name= "X")
y= tf.constant(housing.target.reshape(-1,1), dtype= tf.float32, name="y")

XT =tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()


In [62]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

Compare with Numpy

In [66]:
reset_graph()
X = housing_data_plus_bias
y= housing.target.reshape(-1,1)

theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


Scikit-Learn



In [67]:
from sklearn.linear_model import LinearRegression

In [69]:
lin_reg = LinearRegression()

lin_reg.fit(housing.data, housing.target.reshape(-1,1))

print(np.r_[lin_reg.intercept_.reshape(-1,1), lin_reg.coef_.T])

[[ -3.69419202e+01]
 [  4.36693293e-01]
 [  9.43577803e-03]
 [ -1.07322041e-01]
 [  6.45065694e-01]
 [ -3.97638942e-06]
 [ -3.78654265e-03]
 [ -4.21314378e-01]
 [ -4.34513755e-01]]


In [72]:
#example of np.r_

V = np.array([1,2,3,4,5,6])
Y= np.array([7,8,9,10,11,12])
np.r_[V[0:2],Y[0]]

array([1, 2, 7])

# Using batch Gradient Descent


In [79]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_housing_data = scaler.fit_transform(housing.data)

print(scaled_housing_data)
print(housing.data)

[[ 2.34476576  0.98214266  0.62855945 ..., -0.04959654  1.05254828
  -1.32783522]
 [ 2.33223796 -0.60701891  0.32704136 ..., -0.09251223  1.04318455
  -1.32284391]
 [ 1.7826994   1.85618152  1.15562047 ..., -0.02584253  1.03850269
  -1.33282653]
 ..., 
 [-1.14259331 -0.92485123 -0.09031802 ..., -0.0717345   1.77823747
  -0.8237132 ]
 [-1.05458292 -0.84539315 -0.04021111 ..., -0.09122515  1.77823747
  -0.87362627]
 [-0.78012947 -1.00430931 -0.07044252 ..., -0.04368215  1.75014627
  -0.83369581]]
[[   8.3252       41.            6.98412698 ...,    2.55555556   37.88
  -122.23      ]
 [   8.3014       21.            6.23813708 ...,    2.10984183   37.86
  -122.22      ]
 [   7.2574       52.            8.28813559 ...,    2.80225989   37.85
  -122.24      ]
 ..., 
 [   1.7          17.            5.20554273 ...,    2.3256351    39.43
  -121.22      ]
 [   1.8672       18.            5.32951289 ...,    2.12320917   39.43
  -121.32      ]
 [   2.3886       16.            5.25471698 ...,    2